In [1]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *

# Pracownicy

In [2]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["szef", "sprzedawca", "mechanik", "pomocnik mechanika", "blacharz", 
                      "lakiernik", "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 4), rind(2, 4), rind(1, 3), 1, 
                        1, rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates("2014-01-01", "2018-01-01", np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45), (40, 55),
              (40, 55), (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca[zł/h]"])
for (i, job) in enumerate(jobs_list):
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, birth_date, pesel, city, address, phone, email, employment_date, None, job, wage]

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [3]:
employees_df

,id,imię,nazwisko,data_urodzenia,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca[zł/h]
0,1,Teresa,Dudek,20/01/1977,77012051861,Kielce,ul. Wróżki 91/43,48690561465,teresa.dudek@kuba.pl,2014-01-01,None,szef,74.2
1,2,Zbigniew,Tołpa,29/12/1991,91122911411,Wrocław,ul. Wesoła 89/10,48389401376,zbigniew.tolpa@kuba.pl,2014-01-01,None,sprzedawca,56.9
2,3,Serhiy,Vashchyshyn,03/08/1989,89080352773,Wieliczka,ul. Jana Kochanowskiego 90/5,48113742737,serhiy.vashchyshyn@kuba.pl,2014-01-01,None,sprzedawca,55.3
3,4,Jerzy,Jastrzębski,24/09/1965,65092423355,Wrocław,ul. Sztormowa 65/38,48164568516,jerzy.jastrzebski@kuba.pl,2014-01-01,None,mechanik,47.0
9,5,Ewelina,Szczerbińska,10/04/1962,62041022603,Wrocław,ul. Dzierżona 33/39,48762106712,ewelina.szczerbinska@kuba.pl,2014-01-01,None,diagnosta samochodowy,55.0
4,6,Iwona,Hordyjewicz,18/10/1969,69101871601,Wrocław,ul. Jana Kilińskiego 75,48417165941,iwona.hordyjewicz@kuba.pl,2015-02-07,None,mechanik,47.5
5,7,Beata,Piekarek,07/12/1993,93120758368,Wrocław,ul. Archiwalna 25,48875281328,beata.piekarek@kuba.pl,2015-05-08,None,pomocnik mechanika,38.2
6,8,Łukasz,Dean,08/09/1963,63090894575,Wrocław,ul. Bukowa 3/6,48799680187,lukasz.dean@kuba.pl,2015-08-04,None,pomocnik mechanika,41.1
7,9,Stanisława,Pałyga,23/02/1994,94022330689,Wrocław,ul. Wesoła 13/17,48356617440,stanislawa.palyga@kuba.pl,2015-10-13,None,blacharz,40.7
8,10,Henryka,Rutkowska,17/01/1981,81011764543,Wrocław,ul. Koralowa 80/37,48223624274,henryka.rutkowska@kuba.pl,2015-12-31,None,lakiernik,40.7


# Transakcje

In [4]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [5]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [6]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/5])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
    print(our_cars)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20]
[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20]
[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 14, 15, 17, 18, 19, 20]
[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 14, 15, 17, 19, 20]
[1, 2, 4, 5, 6, 7, 9, 10, 11, 14, 15, 17, 19, 20]
[1, 2, 5, 6, 7, 9, 10, 11, 14, 15, 17, 19, 20]
[1, 2, 6, 7, 9, 10, 11, 14, 15, 17, 19, 20]
[1, 2, 6, 9, 10, 11, 14, 15, 17, 19, 20]
[1, 2, 6, 9, 11, 14, 15, 17, 19, 20]
[1, 2, 6, 9, 11, 15, 17, 19, 20]
[2, 6, 9, 11, 15, 17, 19, 20]
[2, 6, 9, 11, 15, 17, 19]
[2, 6, 9, 11, 17, 19]
[6, 9, 11, 17, 19]
[6, 9, 17, 19]
[6, 17, 19]
[17, 19]
[17]
[]
[np.int64(21)]
[np.int64(21)]
[]
[]
[np.int64(24)]
[]
[np.int64(25)]
[]
[]
[np.int64(27)]
[np.int64(27), np.int64(28)]
[np.int64(28)]
[]
[np.int64(29)]
[]
[np.int64(30)]
[]
[np.int64(31)]
[np.int64(31), np.int64(32)]
[np.int64(32)]
[]
[np.int64(33)]
[np.int64(33), np.int64(34)]
[np.int64(34)]
[]
[np.int

In [7]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [8]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

# klienci

In [9]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        pesel, birth_date = random_pesel_date_man(year_lims)
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        pesel, birth_date = random_pesel_date_woman(year_lims)
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]

In [10]:
clients_df

,id,imię,nazwisko,data_urodzenia,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Łukasz,Ostrowski,16/05/1993,M,93051612096,Opole,ul. Graniczna 95/33,False,48320531128,l_os1993@onet.pl
1,2,Piotr,Kośka,22/11/2007,M,07312218132,Wrocław,ul. Rycerska 19/12,True,48341467780,p-koska9484@wp.pl
2,3,Ignacy,Almashi,04/09/2001,M,01290480837,Wrocław,ul. Polana Brzozowa 7/38,False,48183373113,alm-ign01@interia.pl
3,4,Mariana,Gajer,03/01/1997,F,97010307566,Węgorzewo,ul. Konwaliowa 73,False,48642923885,z0407j24qh@opayq.com
4,5,Szymon,Szremski,13/09/2001,M,01291316636,Wrocław,ul. Jesionowa 76/10,False,48113400236,szrszy2001@wp.pl
...,...,...,...,...,...,...,...,...,...,...,...
1505,1506,Bogdan,Rudniak,08/10/1992,M,92100871235,Wrocław,ul. Tęczowa 68/43,False,48565784361,bog-rudnia@wp.pl
1506,1507,Wojciech,Utracik,23/06/1995,M,95062320359,Wrocław,ul. Gumniska 51/8,False,48998157393,w-utracik95@gmail.com
1507,1508,Joanna,Mistarz,04/10/1986,F,86100433427,Wrocław,ul. Gajewska 84/47,False,48275637059,sd47m3ip312wcj@wp.pl
1508,1509,Jan,Karliński,02/11/1997,M,97110254373,Knurów,ul. Polna 6,False,48500627890,jkarlinski97@gmail.com


In [11]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(72)

# auta

In [21]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()


In [29]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = (i+1 not in car_ids_sell and i+1 in car_ids_buy)
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemość_silnika', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [34]:
df_samochody = create_car_table(max(car_ids)-1)

In [35]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemość_silnika,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Ford,EcoSport,4,2005,Sedan,False,czerwony,Benzyna,1.9,Manualna,5,2,False
1,2,Hero,Honda Passion Plus,2,1994,,True,brązowy,Benzyna,0.3,Automatyczna,0,1,False
2,3,SEAT,Mii,4,2009,SUV,False,biały,Benzyna,1.2,Manualna,5,2,False
3,4,BMW,116,4,2005,Kombi,False,szary,Diesel,1.6,Manualna,5,5,False
4,5,Audi,A7,4,2011,Hatchback,False,czarny,Benzyna,5.1,Automatyczna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1066,Honda,Jazz,4,2000,SUV,False,szary,Benzyna,1.5,Manualna,5,5,False
1066,1067,Hyundai,i30,4,2011,Sedan,False,żółty,Diesel,1.9,Manualna,5,2,False
1067,1068,Honda,CB Unicorn 150,2,2007,,True,szary,Benzyna,0.8,Manualna,0,1,True
1068,1069,Toyota,Auris,4,2006,Sedan,False,niebieski,Benzyna,2.1,Manualna,5,2,False
